In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import pandas as pd
import os

# Kiểm tra xem có GPU không
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Định nghĩa các phép biến đổi cho dữ liệu
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # AlexNet yêu cầu kích thước ảnh 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize theo ImageNet
])

# Tạo dataset và DataLoader cho tập huấn luyện
train_dataset = datasets.ImageFolder(root='/kaggle/input/ai-training-challenge-hutech-orange-classifier/old_oranges_data_1/old_oranges_data/train_set', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Tạo dataset và DataLoader cho tập kiểm tra
test_dataset = datasets.ImageFolder(root='/kaggle/input/ai-training-challenge-hutech-orange-classifier/old_oranges_data_1/old_oranges_data/test_set', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Tải mô hình AlexNet với trọng số pre-trained
alexnet = models.alexnet(weights=models.AlexNet_Weights.IMAGENET1K_V1)

# Điều chỉnh số lớp đầu ra cho bài toán phân loại nhị phân
alexnet.classifier[6] = nn.Linear(alexnet.classifier[6].in_features, 2)

# Chuyển mô hình sang GPU nếu có
alexnet = alexnet.to(device)

# Định nghĩa hàm mất mát và optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(alexnet.parameters(), lr=0.001, momentum=0.9)

# Huấn luyện mô hình
num_epochs = 10
for epoch in range(num_epochs):
    alexnet.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = alexnet(images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}')

# Đánh giá mô hình trên tập kiểm tra
alexnet.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = alexnet(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on test set: {accuracy:.2f}%')

# Tạo DataFrame để lưu kết quả
results = []

# Dự đoán nhãn cho các ảnh trong tập kiểm tra
alexnet.eval()
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = alexnet(images)
        _, predicted = torch.max(outputs.data, 1)
        
        for i in range(len(predicted)):
            image_name = os.path.basename(test_dataset.imgs[i][0])
            label = predicted[i].item()
            results.append([image_name, label])

# Lưu kết quả vào file CSV
df = pd.DataFrame(results, columns=['image_name', 'label'])
df.to_csv('/kaggle/working/submission.csv', index=False)
print("Results saved to submission.csv")

Using device: cuda


Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:01<00:00, 192MB/s]


Epoch 1/10, Loss: 0.15364099351311292
Epoch 2/10, Loss: 0.041760319841357516
Epoch 3/10, Loss: 0.02055201380415752
Epoch 4/10, Loss: 0.01629690892540553
Epoch 5/10, Loss: 0.00345544746981603
Epoch 6/10, Loss: 0.0016484247608404721
Epoch 7/10, Loss: 0.0014211152796686265
Epoch 8/10, Loss: 0.0010162599401287414
Epoch 9/10, Loss: 0.0009836612007347867
Epoch 10/10, Loss: 0.0007621453685820545
Accuracy on test set: 76.00%
Results saved to submission.csv
